In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
import time

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#Add image and video file directories 
os.listdir("test_images/")
os.listdir("test_images_output/")
os.listdir("test_videos/")

#allow inline video play
%matplotlib inline

#create global array for storying slope and intercept data
global full_right_line_slope_array
global full_right_line_intercept_array
global full_left_line_slope_array
global full_left_line_intercept_array


In [ ]:
###
### Define all function blocks used in the lane detection pipeline
###
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform to find edges"""
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=4):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  

    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.

    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """

    line = ''
    i = 1
    for line in lines:
        left = 0
        right = 0
        #print(i) 
        for x1,y1,x2,y2 in line:
            
            #calculate slope of detected line
            m = ((y2-y1)/(x2-x1))
            print('slope m = ' + str(m))
            
            #calculate y-axis intercept of detected line
            #y=mx+b
            b = (y1-((m)*(x1)))
            
            ###
            ### determine if line is right or left portion of lane
            ### y-axis is inverted so positive slope is right side and negative slope is left side
            ###
            
            #right portion of lane
            if m > 0:
                    #append array for slope of right lines in the current frame with the current slope
                    full_right_line_slope_array.append(m)
                    #append array for y-axis intercept of right lines in the current frame with the current intercept
                    full_right_line_intercept_array.append(b)
                    print('right')
                    i+=1
            
            #left portion of lane
            else:
                    #append array for slope of left lines in the current frame with the current slope
                    full_left_line_slope_array.append(m)
                    #append array for y-axis intercept of left lines in the current frame with the current intercept
                    full_left_line_intercept_array.append(b)
                    left= 1
                    print('left')
                    i+=1
    
    #call function to interpolate right and left lines based on averaged slope and intercept of all right and left 
    #line detected in the frame
    average_lines(img,full_right_line_slope_array,full_right_line_intercept_array,full_left_line_slope_array,full_left_line_intercept_array)
    
def average_lines(img,full_right_line_slope_array,full_right_line_intercept_array,full_left_line_slope_array,full_left_line_intercept_array):
       
    color=[255, 0, 0]
    thickness=8
    
    #y=mx+b
    #height of the lines is restricted to the highest and lowest points in the region of interest
    y1 = 540
    y2 = 330
    
    #compute the right line x-coordinate using the average slope and average intercept of the right lines detected 
    x1r = int(((y1 - np.mean(full_right_line_intercept_array))/(np.mean(full_right_line_slope_array))))
    x2r = int(((y2 - np.mean(full_right_line_intercept_array))/(np.mean(full_right_line_slope_array))))
    print('point 1: [' + str(x1r) + ', '  + str(y1) + ']')
    print('point 2: [' + str(x2r) + ', '  + str(y2) + ']')
    cv2.line(img, (x1r, y1), (x2r,y2), color, thickness)
        
    #compute the left line x-coordinate using the average slope and average intercept of the left lines detected
    x1L = int(((y1 - np.mean(full_left_line_intercept_array))/(np.mean(full_left_line_slope_array))))
    x2L = int(((y2 - np.mean(full_left_line_intercept_array))/(np.mean(full_left_line_slope_array))))
    print('point 1: [' + str(x1L) + ', '  + str(y1) + ']')
    print('point 2: [' + str(x2L) + ', '  + str(y2) + ']')
    cv2.line(img, (x1L,y1), (x2L,y2), color, thickness)    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform
    Used to detect lines within an image
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)




In [ ]:
def process_image(image):
        #printing out some stats and plotting
        print('This image is:', type(image), 'with dimensions:', image.shape[0])

        #compute gray scale image
        gray = grayscale(image)
        #useful for debug
        #plt.figure()
        #plt.imshow(gray, cmap='gray')
        #plt.show

        #remove the noise from the computed grayscaled image
        #kernel size determines how much pixel averaging is done. It must be positive and odd.
        #Larger kernel size will take longer to compute
        kernel_size = 9;
        gaussian_gray = gaussian_blur(gray, kernel_size)
        #useful for debug
        #plt.figure
        #plt.imshow(gaussian_gray, cmap='gray')
        #plt.show

        #Detect edges within grayscaled and noise filtered image
        #low threshold = > pixel gradients below this value will not be included as edges
        #high threshold => pixel gradients above this value will be classified as strong edges
        #pixels inbetween the low and high thresholds will be included as long as they touch a strong edge
        low_threshold = 100;
        high_threshold = 150;
        canny_gray = canny(gaussian_gray, low_threshold, high_threshold)
        #useful for debug
        #plt.figure()
        #plt.imshow(canny_gray, cmap='gray')
        #plt.show

        #select the region of interest only
        #selecting a trapazoid with four vertices focused around the center lane
        vertices = np.array([[[100,550],[475,315],[500,315],[900,550]]], dtype=np.int32);
        region_image = region_of_interest(canny_gray, vertices)
        #useful for debug
        plt.imshow(region_image)

        #specify the type of line we are looking for
        rho = 1;              #distance of grid space
        theta = np.pi/180;    #angular resolution of grid space
        threshold = 20;       #parameter specifies the minimum number of votes a candidate line needs to have to make it into the output
        min_line_len = 20;    #minimum line length
        max_line_gap = 300;   #maximum gap in the line

        #draw the averaged and interpolated lines
        line_image = hough_lines(region_image, rho, theta, threshold, min_line_len, max_line_gap)
        #useful for debug
        #plt.figure()
        #plt.imshow(line_image)
        #plt.show

        #compute the composite image of the averaged and interpolated lines on the original image
        composite_image = weighted_img(image, line_image)
        #plt.figure()
        #plt.imshow(composite_image)
        #plt.show
        
        return composite_image    

In [ ]:
full_right_line_slope_array = []
full_right_line_intercept_array = []
full_left_line_slope_array = []
full_left_line_intercept_array = []

white_output = 'test_videos/SolidWhiteRight_Yaman_Output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds

#useful for debug
#clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4").subclip(2,5)
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,10)

#clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")


white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""<video width="960" height="540" controls><source src="{0}"></video>""".format(white_output))
    

    
